In [0]:
access_key = 'your_key'
secret_key = 'your_secret_key'
encoded_secret_key = secret_key.replace("/", "%2F")
aws_bucket_name = "your_pipe_line"
mount_name = "s3dataread"

dbutils.fs.mount(f"s3a://{access_key}:{encoded_secret_key}@{aws_bucket_name}", f"/mnt/{mount_name}")
display(dbutils.fs.ls(f"/mnt/{mount_name}"))

path,name,size,modificationTime
dbfs:/mnt/s3dataread/raw_data/,raw_data/,0,0


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, BooleanType, DateType, DecimalType
from pyspark.sql.functions import col, when, sum, avg, row_number,count ,round,countDistinct,dense_rank,rank
from pyspark.sql.window import Window

In [0]:
from pyspark.sql import SparkSession 

#create session
spark = SparkSession.builder.appName("Olympic Data Analysis").getOrCreate()

In [0]:
%sh ls /mnt/s3dataread/raw_data/

ls: cannot access '/mnt/s3dataread/raw_data/': No such file or directory


In [0]:
dbutils.fs.mounts() 

Out[25]: [MountInfo(mountPoint='/mnt/s3dataread', source='s3a://olympic-data-pipe-tanmay', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType='sse-s3'),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType='sse-s3'),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType='sse-s3'),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType='sse-s3')]

In [0]:
coaches_file = "dbfs:/mnt/s3dataread/raw_data/Coaches.csv"
athletes_file = "dbfs:/mnt/s3dataread/raw_data/Athletes.csv"
entriesgender_file = "dbfs:/mnt/s3dataread/raw_data/EntriesGender.csv"
medals_file = "dbfs:/mnt/s3dataread/raw_data/Medals.csv"
teams_file = "dbfs:/mnt/s3dataread/raw_data/Teams.csv"

In [0]:

coaches = spark.read.csv(coaches_file, inferSchema = True, header= True)
athletes = spark.read.csv(athletes_file, inferSchema = True, header= True)
entriesgender = spark.read.csv(entriesgender_file, inferSchema = True, header= True)
medals = spark.read.csv(medals_file, inferSchema = True, header= True)
teams = spark.read.csv(teams_file, inferSchema = True, header= True)

In [0]:
coaches.show(10)

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
+--------------------+--------------------+-----------------+--------+
only s

In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
athletes.show(10)

+-----------------+--------------------+-------------------+
|       PersonName|             Country|         Discipline|
+-----------------+--------------------+-------------------+
|  AALERUD Katrine|              Norway|       Cycling Road|
|      ABAD Nestor|               Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|               Italy|             Rowing|
|   ABALDE Alberto|               Spain|         Basketball|
|    ABALDE Tamara|               Spain|         Basketball|
|        ABALO Luc|              France|           Handball|
|     ABAROA Cesar|               Chile|             Rowing|
|    ABASS Abobakr|               Sudan|           Swimming|
| ABBASALI Hamideh|Islamic Republic ...|             Karate|
|    ABBASOV Islam|          Azerbaijan|          Wrestling|
+-----------------+--------------------+-------------------+
only showing top 10 rows



In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
entriesgender.show(10)

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
|          Badminton|    86|  87|  173|
|  Baseball/Softball|    90| 144|  234|
|         Basketball|   144| 144|  288|
|   Beach Volleyball|    48|  48|   96|
|             Boxing|   102| 187|  289|
+-------------------+------+----+-----+
only showing top 10 rows



In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.show(10)

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
+----+--------------------+----+------+------+-----+-------------+
only showing top 10 rows



In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
teams.show(10,False)

+-----------+--------------+--------------------------+-----+
|TeamName   |Discipline    |Country                   |Event|
+-----------+--------------+--------------------------+-----+
|Belgium    |3x3 Basketball|Belgium                   |Men  |
|China      |3x3 Basketball|People's Republic of China|Men  |
|China      |3x3 Basketball|People's Republic of China|Women|
|France     |3x3 Basketball|France                    |Women|
|Italy      |3x3 Basketball|Italy                     |Women|
|Japan      |3x3 Basketball|Japan                     |Men  |
|Japan      |3x3 Basketball|Japan                     |Women|
|Latvia     |3x3 Basketball|Latvia                    |Men  |
|Mongolia   |3x3 Basketball|Mongolia                  |Women|
|Netherlands|3x3 Basketball|Netherlands               |Men  |
+-----------+--------------+--------------------------+-----+
only showing top 10 rows



In [0]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



Exploratory Data Analysis

In [0]:
#List all the unique disciplines present in the athletes' data.
athletes.select("Discipline").distinct().show()

+-------------------+
|         Discipline|
+-------------------+
|             Boxing|
|             Rowing|
|  Baseball/Softball|
|               Judo|
|           Swimming|
|         Basketball|
|           Handball|
|Rhythmic Gymnastics|
|             Karate|
|          Badminton|
|          Athletics|
|           Shooting|
|Artistic Gymnastics|
|       Cycling Road|
|          Wrestling|
|       Table Tennis|
|          Taekwondo|
|      Weightlifting|
|  Artistic Swimming|
|            Fencing|
+-------------------+
only showing top 20 rows



In [0]:
#Find the total number of coaches for each country.
coaches.groupBy('Country').count().orderBy("count", ascending=False).show(20, False)

+--------------------------+-----+
|Country                   |count|
+--------------------------+-----+
|Japan                     |35   |
|Spain                     |28   |
|United States of America  |28   |
|Australia                 |22   |
|Canada                    |16   |
|Italy                     |14   |
|People's Republic of China|12   |
|Egypt                     |12   |
|South Africa              |12   |
|ROC                       |12   |
|Argentina                 |11   |
|France                    |10   |
|Venezuela                 |10   |
|Netherlands               |10   |
|Nigeria                   |9    |
|Germany                   |9    |
|Mexico                    |8    |
|New Zealand               |8    |
|Great Britain             |7    |
|Republic of Korea         |7    |
+--------------------------+-----+
only showing top 20 rows



In [0]:
#Identify the top 5 countries with the highest number of gold medals.
medals.orderBy(col("Gold").desc()).select('Team_Country','Gold').show(5, False)

+--------------------------+----+
|Team_Country              |Gold|
+--------------------------+----+
|United States of America  |39  |
|People's Republic of China|38  |
|Japan                     |27  |
|Great Britain             |22  |
|ROC                       |20  |
+--------------------------+----+
only showing top 5 rows



Deep Dive into Analysis

In [0]:
#Find the top 3 countries with the highest average number of medals per athlete. Consider all types of medals (gold, silver, bronze) in the calculation.
country_atheletes = athletes.groupBy('Country').agg(count('PersonName').alias("athlete_Count"))
medals_and_athletes = medals.join(country_atheletes, medals.Team_Country == country_atheletes.Country, "inner")

In [0]:
medals_and_athletes = medals_and_athletes.withColumn('Avg_Medals_Per_Athlete', round(col('Total')/col('athlete_Count'),3))
medals_and_athletes.orderBy(col('Avg_Medals_Per_Athlete').desc()).select('Country','Total','athlete_Count','Avg_Medals_Per_Athlete').show(3)

+----------+-----+-------------+----------------------+
|   Country|Total|athlete_Count|Avg_Medals_Per_Athlete|
+----------+-----+-------------+----------------------+
|San Marino|    3|            4|                  0.75|
|   Bermuda|    1|            2|                   0.5|
|   Armenia|    4|           16|                  0.25|
+----------+-----+-------------+----------------------+
only showing top 3 rows



In [0]:
#Determine the top 5 countries based on a weighted score, where gold medals are worth 3 points, silver medals are worth 2 points, and bronze medals are worth 1 point.
weighted_scores = medals.withColumn(
    "Weighted_Score", col("Gold") * 3 + col("Silver") * 2 + col("Bronze")
).groupBy("Team_Country").agg(
    sum("Weighted_Score").alias("Total_Score")
).orderBy(col("Total_Score").desc())

weighted_scores.show(5,False)



+--------------------------+-----------+
|Team_Country              |Total_Score|
+--------------------------+-----------+
|United States of America  |232        |
|People's Republic of China|196        |
|ROC                       |139        |
|Great Britain             |130        |
|Japan                     |126        |
+--------------------------+-----------+
only showing top 5 rows



In [0]:
#For each country, find the coach with the most number of disciplines coached. If there is a tie, include all relevant coaches.
coach_discipline_count = coaches.groupBy("Country", "Name").agg(
    countDistinct("Discipline").alias("Num_Disciplines")
)

In [0]:
#the top 3 disciplines with the highest ratio of female to male participants.
ratio_female_to_male = entriesgender.withColumn(
    "Female_Male_Ratio", col("Female") / col("Male")
).orderBy(col("Female_Male_Ratio").desc()).limit(3)

ratio_female_to_male.show()


+--------------------+------+----+-----+------------------+
|          Discipline|Female|Male|Total| Female_Male_Ratio|
+--------------------+------+----+-----+------------------+
|Cycling BMX Frees...|    10|   9|   19|1.1111111111111112|
|              Diving|    72|  71|  143|1.0140845070422535|
|      3x3 Basketball|    32|  32|   64|               1.0|
+--------------------+------+----+-----+------------------+



In [0]:
#the countries that have never won a gold medal but have won at least one silver and one bronze medal.
no_gold_countries = medals.filter(
    (col("Gold") == 0) & (col("Silver") > 0) & (col("Bronze") > 0)
).select("Team_Country")

no_gold_countries.show()


+------------------+
|      Team_Country|
+------------------+
|          Colombia|
|        Azerbaijan|
|Dominican Republic|
|           Armenia|
|        Kyrgyzstan|
|          Mongolia|
|         Argentina|
|        San Marino|
|            Jordan|
|          Malaysia|
|           Nigeria|
+------------------+



In [0]:
#country and discipline pair with the highest number of athletes, and list the top 5 such pairs
country_discipline_athletes = athletes.groupBy("Country", "Discipline").count().alias("Num_Athletes")
top_5_pairs = country_discipline_athletes.orderBy(col("count").desc()).limit(5)

top_5_pairs.show()


+--------------------+----------+-----+
|             Country|Discipline|count|
+--------------------+----------+-----+
|United States of ...| Athletics|  144|
|             Germany| Athletics|   95|
|       Great Britain| Athletics|   75|
|               Italy| Athletics|   73|
|               Japan| Athletics|   70|
+--------------------+----------+-----+

